# Tutorial: Training a sound classifier

## Prerequisite

Complete running [Data-Preprocessing.ipynb](Data-Preprocessing.ipynb) to preprocess raw audio files.

In [1]:
# Comment out if your don't have my library dl-cliche, or you can install by `pip install dl-cliche`. It's good for you :)
# This will make your notebook work easier (enables auto-reload, and etc ...)
from dlcliche.notebook import *
import warnings
warnings.simplefilter('ignore')

In [2]:
# Essential PyTorch
import torch
import torchaudio

# Other modules used in this notebook
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import Audio
import yaml
from easydict import EasyDict
from sklearn.model_selection import train_test_split

In [3]:
### CONFIGURE YOURS ###
FSD2018_ROOT = Path('/data/A/2018fsd')
WORK = Path('work')

# Load configuration parameters
with open('config.yaml') as conf:
    cfg = EasyDict(yaml.safe_load(conf))

print(cfg)

{'clip_length': 1.0, 'sample_rate': 44100, 'hop_length': 441, 'n_fft': 1024, 'n_mels': 64, 'f_min': 0, 'f_max': 22050}


## FSDKaggle2018 meta data to files/labels

In [4]:
# Make lists of filenames and labels from meta files
filenames, labels = {}, {}
for split, npy_folder, meta_filename in [['train', 'work/FSDKaggle2018.audio_train', 'train_post_competition.csv'],
                                                                     ['test', 'work/FSDKaggle2018.audio_test', 'test_post_competition_scoring_clips.csv']]:
    df = pd.read_csv(FSD2018_ROOT/'FSDKaggle2018.meta'/meta_filename)
    filenames[split] = [(npy_folder + '/' + fname.replace('.wav', '.npy')) for fname in df.fname.values]
    labels[split] = list(df.label.values)

# Make a list of classes, converting labels into numbers
classes = sorted(set(labels['train'] + labels['test']))
for split in labels:
    labels[split] = [classes.index(label) for label in labels[split]]

print(filenames['train'][:3])
print(labels['train'][:3])
print(len(classes), classes[:3])

['work/FSDKaggle2018.audio_train/00044347.npy', 'work/FSDKaggle2018.audio_train/001ca53d.npy', 'work/FSDKaggle2018.audio_train/002d256b.npy']
[23, 30, 38]
41 ['Acoustic_guitar', 'Applause', 'Bark']


## Split training set into training/validation sets

In [5]:
filenames['train'], filenames['valid'], labels['train'], labels['valid'] = train_test_split(
     filenames['train'],  labels['train'], test_size=0.1, random_state=42)

print(f'Number of train/val files are = {len(filenames["train"])}/{len(filenames["valid"])}')
print('First 3 samples assigned to validation set:')
print(filenames['valid'][:3])
print(labels['valid'][:3])

Number of train/val files are = 8525/948
First 3 samples assigned to validation set:
['work/FSDKaggle2018.audio_train/60f9e3ed.npy', 'work/FSDKaggle2018.audio_train/3e20cc09.npy', 'work/FSDKaggle2018.audio_train/e3278076.npy']
[17, 37, 1]


## Dataset class and dataloaders

In [6]:
def sample_length(log_mel_spec):
    return log_mel_spec.shape[-1]


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, filenames, labels, transforms=None):
        assert len(filenames) == len(labels), f'Inconsistent length of filenames and labels.'

        self.filenames = filenames
        self.labels = labels
        self.transforms = transforms

        # Calculate length of clip this dataset will make
        self.sample_length = int((cfg.clip_length * cfg.sample_rate + cfg.hop_length - 1) // cfg.hop_length)

        # Test with first file
        assert self[0][0].shape[-1] == self.sample_length, f'Check your files, failed to load {filenames[0]}'

        # Show basic info.
        print(f'Dataset will yield log-mel spectrogram {len(self)} data samples in shape [1, {cfg.n_mels}, {self.sample_length}]')

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        assert 0 <= index and index < len(self)
        
        log_mel_spec = np.load(self.filenames[index])

        # Padding if sample is shorter than expected - both head & tail are filled with 0s
        pad_size = self.sample_length - sample_length(log_mel_spec)
        if pad_size > 0:
            offset = pad_size // 2
            log_mel_spec = np.pad(log_mel_spec, ((0, 0), (0, 0), (offset, pad_size - offset)), 'constant')

        # Random crop
        crop_size = sample_length(log_mel_spec) - self.sample_length
        if crop_size > 0:
            start = np.random.randint(0, crop_size)
            log_mel_spec = log_mel_spec[..., start:start + self.sample_length]

        # Apply augmentations
        if self.transforms is not None:
            log_mel_spec = self.transforms(log_mel_spec)

        return torch.Tensor(log_mel_spec), self.labels[index]


train_dataset = MyDataset(cfg, filenames['train'], labels['train'])
valid_dataset = MyDataset(cfg, filenames['valid'], labels['valid'])
test_dataset = MyDataset(cfg, filenames['test'], labels['test'])

Dataset will yield log-mel spectrogram 8525 data samples in shape [1, 64, 100]
Dataset will yield log-mel spectrogram 948 data samples in shape [1, 64, 100]
Dataset will yield log-mel spectrogram 1600 data samples in shape [1, 64, 100]


In [7]:
BS = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=False, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BS, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, pin_memory=True)

## Model definition

Borrowed from MNIST example.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# Mostly borrowed from https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self, n_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.conv4 = nn.Conv2d(128, 256, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(73216, 128)
        self.fc2 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Training step definition

Using PyTorch Lightning. This is also from a example, [MNIST hello world
](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/01-mnist-hello-world.ipynb).

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy


class MyLearner(pl.LightningModule):

    def __init__(self, n_classes, learning_rate=3e-4):

        super().__init__()
        self.learning_rate = learning_rate

        #self.model = Cnn14Mel(n_classes)
        self.model = Net(n_classes)

    def forward(self, x):
        return self.model(x)

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx, split='val'):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        self.log(f'{split}_loss', loss, prog_bar=True)
        self.log(f'{split}_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx, split='test')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return valid_loader

    def test_dataloader(self):
        return test_loader

In [10]:
learner = MyLearner(n_classes=len(classes))
checkpoint = pl.callbacks.ModelCheckpoint(monitor='val_acc')
trainer = pl.Trainer(gpus=1, max_epochs=100, callbacks=[checkpoint])
trainer.fit(learner)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | Net  | 9 M   


1

In [11]:
learner.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])
trainer.test()

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7319, device='cuda:0'),
 'test_loss': tensor(1.0282, device='cuda:0'),
 'val_acc': tensor(0.7278, device='cuda:0'),
 'val_loss': tensor(1.1660, device='cuda:0')}
--------------------------------------------------------------------------------



[{'val_loss': 1.1660009622573853,
  'val_acc': 0.7278481125831604,
  'test_loss': 1.0282281637191772,
  'test_acc': 0.7318750023841858}]